In [2]:
import os
os.chdir("/home/ziyan/Desktop/Competition/")

In [4]:
import torch
torch.cuda.is_available()

True

In [4]:
import json
import sys
# sys.path.append("drive/My Drive/Learning-to-Drive_Competition/starter_kit")
from dataset import Drive360Loader

# load the config.json file that specifies data 
# location parameters and other hyperparameters 
# required.
config = json.load(open('./config.json'))

# create a train, validation and test data loader
train_loader = Drive360Loader(config, 'train')
validation_loader = Drive360Loader(config, 'validation')
test_loader = Drive360Loader(config, 'test')

# print the data (keys) available for use. See full 
# description of each data type in the documents.
print('Loaded train loader with the following data available as a dict.')
print(train_loader.drive360.dataframe.keys())


Phase: train # of data: 38328
Phase: validation # of data: 2522
Phase: test # of data: 6859
Loaded train loader with the following data available as a dict.
Index(['cameraRight', 'cameraFront', 'cameraRear', 'cameraLeft', 'canSteering',
       'canSpeed', 'chapter'],
      dtype='object')


In [4]:
test_loader.drive360.dataframe['cameraRight'].iloc[0]

'Egg/go_pro_1/image/6/img00040.jpg'

In [5]:
# list(models.resnet34(pretrained=True).children())

In [5]:
""" 6. resnet18 + front + rear lstm """

from torchvision import models
import torch.nn as nn
import torch

class SomeDrivingModel(nn.Module):
    def __init__(self):
        super(SomeDrivingModel, self).__init__()
        final_concat_size = 0
        
        # Main CNN
        cnn = models.resnet18(pretrained=True)
        self.features = nn.Sequential(*list(cnn.children())[:-1])
        self.intermediate = nn.Sequential(nn.Linear(
                          cnn.fc.in_features, 128),
                          nn.ReLU())
        final_concat_size += 128

        # Main LSTM
        self.lstm = nn.LSTM(input_size=128,
                            hidden_size=64,
                            num_layers=3,
                            batch_first=False)
        final_concat_size += 64
        
#         # Right CNN
#         cnn_right = models.resnet18(pretrained=True)
#         self.features_right = nn.Sequential(*list(cnn_right.children())[:-1])
#         self.intermediate_right = nn.Sequential(nn.Linear(
#                           cnn_right.fc.in_features, 128),
#                           nn.ReLU())
#         final_concat_size += 128
        
#         # Right LSTM
#         self.lstm_right = nn.LSTM(input_size=128,
#                             hidden_size=64,
#                             num_layers=3,
#                             batch_first=False)
#         final_concat_size += 64
        
#         # Left CNN
#         cnn_left = models.resnet18(pretrained=True)
#         self.features_left = nn.Sequential(*list(cnn_left.children())[:-1])
#         self.intermediate_left = nn.Sequential(nn.Linear(
#                           cnn_left.fc.in_features, 128),
#                           nn.ReLU())
#         final_concat_size += 128
        
#         # Left LSTM
#         self.lstm_left = nn.LSTM(input_size=128,
#                             hidden_size=64,
#                             num_layers=3,
#                             batch_first=False)
#         final_concat_size += 64
        
#         # Rear CNN
#         cnn_rear = models.resnet18(pretrained=True)
#         self.features_rear = nn.Sequential(*list(cnn_rear.children())[:-1])
#         self.intermediate_rear = nn.Sequential(nn.Linear(
#                           cnn_rear.fc.in_features, 128),
#                           nn.ReLU())
#         final_concat_size += 128
        
        # rear LSTM
        self.lstm_rear = nn.LSTM(input_size=128,
                            hidden_size=64,
                            num_layers=3,
                            batch_first=False)
        final_concat_size += 64
        
        # Angle Regressor
        self.control_angle = nn.Sequential(
            nn.Linear(final_concat_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
        # Speed Regressor
        self.control_speed = nn.Sequential(
            nn.Linear(final_concat_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
    
    def forward(self, data):
        module_outputs = []
        lstm_i = []  # cameraFront
        lstm_r = []  # cameraRight
        lstm_l = []  # cameraLeft
        lstm_rear = []  # cameraRear
        
        # Loop through temporal sequence of
        # front facing camera images and pass 
        # through the cnn.
        for k, v in data['cameraFront'].items():
            v = v.cuda()
            x = self.features(v)
            x = x.view(x.size(0), -1)
            x = self.intermediate(x)
            lstm_i.append(x)
            # feed the current front facing camera
            # output directly into the 
            # regression networks.
            if k == 0:
                module_outputs.append(x)
        
#         for k, v in data['cameraRight'].items():
#             v = v.cuda()
#             x = self.features_right(v)
#             x = x.view(x.size(0), -1)
#             x = self.intermediate_right(x)
#             lstm_r.append(x)
#             if k == 0:
#                 module_outputs.append(x)
        
#         for k, v in data['cameraLeft'].items():
#             v = v.cuda()
#             x = self.features_left(v)
#             x = x.view(x.size(0), -1)
#             x = self.intermediate_left(x)
#             lstm_l.append(x)
#             if k == 0:
#                 module_outputs.append(x)
        
        for k, v in data['cameraRear'].items():
            v = v.cuda()
#             x = self.features_rear(v)
#             x = x.view(x.size(0), -1)
#             x = self.intermediate_rear(x)
            x = self.features(v)
            x = x.view(x.size(0), -1)
            x = self.intermediate(x)
            lstm_rear.append(x)
#             if k == 0:
#                 module_outputs.append(x)

        # Feed temporal outputs of CNN into LSTM
        i_lstm, _ = self.lstm(torch.stack(lstm_i))
        module_outputs.append(i_lstm[-1])
        
#         r_lstm, _ = self.lstm_right(torch.stack(lstm_r))
#         module_outputs.append(r_lstm[-1])
        
#         l_lstm, _ = self.lstm_left(torch.stack(lstm_l))
#         module_outputs.append(l_lstm[-1])
        
        rear_lstm, _ = self.lstm_rear(torch.stack(lstm_rear))
        module_outputs.append(rear_lstm[-1])
        
        # Concatenate current image CNN output
        # and LSTM output.
        x_cat = torch.cat(module_outputs, dim=-1)
        
        # Feed concatenated outputs into the 
        # regession networks.
        prediction = {'canSteering': torch.squeeze(self.control_angle(x_cat)),
                      'canSpeed': torch.squeeze(self.control_speed(x_cat))}
        return prediction

# Create your own driving model, this is
#  a very basic one. 
model = SomeDrivingModel().cuda()

In [7]:
import torch.optim as optim
from time import time
import gc

criterion1 = nn.SmoothL1Loss()
criterion2 = nn.SmoothL1Loss()

optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
model.train()
epoch_error = {}

for epoch in range(10):
    running_loss = 0.0
    start_time = time()
    epoch_error[epoch] = []
    
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        prediction = model(data)
        
        loss = criterion1(prediction['canSpeed'].cuda(), target['canSpeed'].cuda()) \
                + criterion2(prediction['canSteering'].cuda(), target['canSteering'].cuda())
        
#         loss = criterion1(prediction['canSpeed'].cuda(), target['canSpeed'].cuda())
        
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if batch_idx % 2 == 1:
            cost_time = time() - start_time
          
            print('[epoch: %d, batch:  %5d] loss: %.5f  | %.2f' %
                  (epoch + 1, batch_idx + 1, running_loss / 2.0, cost_time))
            
            running_loss = 0.0
            start_time = time()
        
        epoch_error[epoch].append(loss)
        
        # Remove this when actually training. 
        # Used to terminate early. 
        if batch_idx >= 5000: 
            break
            
    gc.collect()

[epoch: 1, batch:      2] loss: 0.53716  | 1.42
[epoch: 1, batch:      4] loss: 0.32949  | 0.19
[epoch: 1, batch:      6] loss: 0.54619  | 0.18
[epoch: 1, batch:      8] loss: 0.60564  | 0.17
[epoch: 1, batch:     10] loss: 0.38309  | 0.17
[epoch: 1, batch:     12] loss: 0.53608  | 0.19
[epoch: 1, batch:     14] loss: 0.40215  | 0.17
[epoch: 1, batch:     16] loss: 1.54246  | 0.17
[epoch: 1, batch:     18] loss: 1.08935  | 0.18
[epoch: 1, batch:     20] loss: 0.43924  | 0.18
[epoch: 1, batch:     22] loss: 1.12192  | 0.18
[epoch: 1, batch:     24] loss: 1.58606  | 0.18
[epoch: 1, batch:     26] loss: 0.19112  | 0.18
[epoch: 1, batch:     28] loss: 0.69837  | 0.18
[epoch: 1, batch:     30] loss: 0.73107  | 0.18
[epoch: 1, batch:     32] loss: 0.59604  | 0.18
[epoch: 1, batch:     34] loss: 0.28533  | 0.18
[epoch: 1, batch:     36] loss: 2.11958  | 0.18
[epoch: 1, batch:     38] loss: 0.10758  | 0.18
[epoch: 1, batch:     40] loss: 0.85731  | 0.18
[epoch: 1, batch:     42] loss: 1.03294 

[epoch: 1, batch:    346] loss: 0.33977  | 0.18
[epoch: 1, batch:    348] loss: 1.13683  | 0.18
[epoch: 1, batch:    350] loss: 0.35701  | 0.18
[epoch: 1, batch:    352] loss: 0.65229  | 0.18
[epoch: 1, batch:    354] loss: 0.80642  | 0.18
[epoch: 1, batch:    356] loss: 0.38529  | 0.18
[epoch: 1, batch:    358] loss: 0.60205  | 0.18
[epoch: 1, batch:    360] loss: 0.61455  | 0.18
[epoch: 1, batch:    362] loss: 0.29691  | 0.18
[epoch: 1, batch:    364] loss: 0.10586  | 0.18
[epoch: 1, batch:    366] loss: 0.44022  | 0.18
[epoch: 1, batch:    368] loss: 0.23240  | 0.18
[epoch: 1, batch:    370] loss: 0.27119  | 0.18
[epoch: 1, batch:    372] loss: 0.58770  | 0.18
[epoch: 1, batch:    374] loss: 0.12137  | 0.18
[epoch: 1, batch:    376] loss: 0.26348  | 0.18
[epoch: 1, batch:    378] loss: 0.40861  | 0.18
[epoch: 1, batch:    380] loss: 1.09203  | 0.18
[epoch: 1, batch:    382] loss: 0.39924  | 0.18
[epoch: 1, batch:    384] loss: 0.07209  | 0.18
[epoch: 1, batch:    386] loss: 0.38561 

[epoch: 1, batch:    690] loss: 0.30213  | 0.18
[epoch: 1, batch:    692] loss: 0.37185  | 0.18
[epoch: 1, batch:    694] loss: 1.33979  | 0.18
[epoch: 1, batch:    696] loss: 0.53798  | 0.18
[epoch: 1, batch:    698] loss: 0.35957  | 0.18
[epoch: 1, batch:    700] loss: 0.43300  | 0.18
[epoch: 1, batch:    702] loss: 0.14358  | 0.18
[epoch: 1, batch:    704] loss: 0.28917  | 0.18
[epoch: 1, batch:    706] loss: 0.22156  | 0.18
[epoch: 1, batch:    708] loss: 0.79058  | 0.18
[epoch: 1, batch:    710] loss: 0.78743  | 0.18
[epoch: 1, batch:    712] loss: 0.50154  | 0.18
[epoch: 1, batch:    714] loss: 1.68402  | 0.18
[epoch: 1, batch:    716] loss: 0.49971  | 0.18
[epoch: 1, batch:    718] loss: 0.58964  | 0.18
[epoch: 1, batch:    720] loss: 0.55928  | 0.18
[epoch: 1, batch:    722] loss: 0.39582  | 0.18
[epoch: 1, batch:    724] loss: 0.12901  | 0.18
[epoch: 1, batch:    726] loss: 0.54810  | 0.18
[epoch: 1, batch:    728] loss: 1.27495  | 0.18
[epoch: 1, batch:    730] loss: 1.42656 

[epoch: 1, batch:   1034] loss: 0.67917  | 0.18
[epoch: 1, batch:   1036] loss: 0.46703  | 0.18
[epoch: 1, batch:   1038] loss: 0.08175  | 0.18
[epoch: 1, batch:   1040] loss: 0.62861  | 0.18
[epoch: 1, batch:   1042] loss: 0.47409  | 0.18
[epoch: 1, batch:   1044] loss: 0.56097  | 0.18
[epoch: 1, batch:   1046] loss: 0.59330  | 0.18
[epoch: 1, batch:   1048] loss: 1.00939  | 0.18
[epoch: 1, batch:   1050] loss: 0.68412  | 0.18
[epoch: 1, batch:   1052] loss: 0.72326  | 0.18
[epoch: 1, batch:   1054] loss: 0.17802  | 0.18
[epoch: 1, batch:   1056] loss: 0.15778  | 0.18
[epoch: 1, batch:   1058] loss: 0.32794  | 0.18
[epoch: 1, batch:   1060] loss: 0.94226  | 0.18
[epoch: 1, batch:   1062] loss: 0.74382  | 0.18
[epoch: 1, batch:   1064] loss: 0.59798  | 0.18
[epoch: 1, batch:   1066] loss: 0.26675  | 0.18
[epoch: 1, batch:   1068] loss: 0.48951  | 0.18
[epoch: 1, batch:   1070] loss: 0.25198  | 0.18
[epoch: 1, batch:   1072] loss: 1.38223  | 0.18
[epoch: 1, batch:   1074] loss: 0.62126 

[epoch: 1, batch:   1378] loss: 0.92520  | 0.18
[epoch: 1, batch:   1380] loss: 0.36455  | 0.18
[epoch: 1, batch:   1382] loss: 0.18323  | 0.18
[epoch: 1, batch:   1384] loss: 0.24614  | 0.18
[epoch: 1, batch:   1386] loss: 0.28829  | 0.18
[epoch: 1, batch:   1388] loss: 0.66898  | 0.18
[epoch: 1, batch:   1390] loss: 0.47189  | 0.18
[epoch: 1, batch:   1392] loss: 0.34402  | 0.18
[epoch: 1, batch:   1394] loss: 0.16135  | 0.18
[epoch: 1, batch:   1396] loss: 0.56814  | 0.18
[epoch: 1, batch:   1398] loss: 0.04072  | 0.18
[epoch: 1, batch:   1400] loss: 0.61122  | 0.18
[epoch: 1, batch:   1402] loss: 0.40381  | 0.18
[epoch: 1, batch:   1404] loss: 0.38727  | 0.18
[epoch: 1, batch:   1406] loss: 0.69135  | 0.18
[epoch: 1, batch:   1408] loss: 0.90668  | 0.18
[epoch: 1, batch:   1410] loss: 0.85228  | 0.18
[epoch: 1, batch:   1412] loss: 0.60331  | 0.18
[epoch: 1, batch:   1414] loss: 0.47959  | 0.18
[epoch: 1, batch:   1416] loss: 0.35140  | 0.18
[epoch: 1, batch:   1418] loss: 0.30645 

...

[epoch: 4, batch:   3884] loss: 0.08098  | 0.18
[epoch: 4, batch:   3886] loss: 0.02679  | 0.18
[epoch: 4, batch:   3888] loss: 1.20751  | 0.18
[epoch: 4, batch:   3890] loss: 0.14083  | 0.18
[epoch: 4, batch:   3892] loss: 0.06822  | 0.18
[epoch: 4, batch:   3894] loss: 0.12028  | 0.19
[epoch: 4, batch:   3896] loss: 0.34219  | 0.18
[epoch: 4, batch:   3898] loss: 0.22087  | 0.18
[epoch: 4, batch:   3900] loss: 0.02377  | 0.18
[epoch: 4, batch:   3902] loss: 0.08003  | 0.18
[epoch: 4, batch:   3904] loss: 0.09119  | 0.18
[epoch: 4, batch:   3906] loss: 0.03211  | 0.18
[epoch: 4, batch:   3908] loss: 0.70380  | 0.18
[epoch: 4, batch:   3910] loss: 0.05595  | 0.19
[epoch: 4, batch:   3912] loss: 0.04443  | 0.18
[epoch: 4, batch:   3914] loss: 0.03738  | 0.17
[epoch: 4, batch:   3916] loss: 0.06517  | 0.18
[epoch: 4, batch:   3918] loss: 0.19837  | 0.19
[epoch: 4, batch:   3920] loss: 0.08448  | 0.18
[epoch: 4, batch:   3922] loss: 0.48568  | 0.18
[epoch: 4, batch:   3924] loss: 0.97499 

[epoch: 4, batch:   4228] loss: 0.11456  | 0.18
[epoch: 4, batch:   4230] loss: 0.06068  | 0.19
[epoch: 4, batch:   4232] loss: 0.14566  | 0.18
[epoch: 4, batch:   4234] loss: 0.03837  | 0.18
[epoch: 4, batch:   4236] loss: 0.07257  | 0.18
[epoch: 4, batch:   4238] loss: 0.17395  | 1.40
[epoch: 4, batch:   4240] loss: 0.11540  | 0.17
[epoch: 4, batch:   4242] loss: 0.71442  | 0.18
[epoch: 4, batch:   4244] loss: 0.27871  | 0.18
[epoch: 4, batch:   4246] loss: 0.76891  | 0.18
[epoch: 4, batch:   4248] loss: 0.54728  | 0.17
[epoch: 4, batch:   4250] loss: 0.18566  | 0.18
[epoch: 4, batch:   4252] loss: 0.20870  | 0.18
[epoch: 4, batch:   4254] loss: 0.20547  | 0.19
[epoch: 4, batch:   4256] loss: 0.08078  | 0.17
[epoch: 4, batch:   4258] loss: 0.08472  | 0.18
[epoch: 4, batch:   4260] loss: 0.37705  | 0.18
[epoch: 4, batch:   4262] loss: 0.09284  | 0.19
[epoch: 4, batch:   4264] loss: 0.51051  | 0.18
[epoch: 4, batch:   4266] loss: 0.03379  | 0.19
[epoch: 4, batch:   4268] loss: 0.17696 

[epoch: 4, batch:   4570] loss: 0.13716  | 0.18
[epoch: 4, batch:   4572] loss: 0.15786  | 0.19
[epoch: 4, batch:   4574] loss: 0.09896  | 0.18
[epoch: 4, batch:   4576] loss: 0.05882  | 0.18
[epoch: 4, batch:   4578] loss: 0.12842  | 0.18
[epoch: 4, batch:   4580] loss: 0.13628  | 0.18
[epoch: 4, batch:   4582] loss: 0.39775  | 0.19
[epoch: 4, batch:   4584] loss: 0.02592  | 0.17
[epoch: 4, batch:   4586] loss: 0.19830  | 0.18
[epoch: 4, batch:   4588] loss: 0.03996  | 0.18
[epoch: 4, batch:   4590] loss: 0.08161  | 0.20
[epoch: 4, batch:   4592] loss: 0.10050  | 0.17
[epoch: 4, batch:   4594] loss: 0.17477  | 0.18
[epoch: 4, batch:   4596] loss: 0.34470  | 0.17
[epoch: 4, batch:   4598] loss: 0.09387  | 0.18
[epoch: 4, batch:   4600] loss: 0.14674  | 0.18
[epoch: 4, batch:   4602] loss: 0.08143  | 0.19
[epoch: 4, batch:   4604] loss: 0.04929  | 0.17
[epoch: 4, batch:   4606] loss: 0.04019  | 0.18
[epoch: 4, batch:   4608] loss: 0.03960  | 0.18
[epoch: 4, batch:   4610] loss: 0.13149 

[epoch: 4, batch:   4912] loss: 0.12679  | 0.18
[epoch: 4, batch:   4914] loss: 0.07131  | 0.19
[epoch: 4, batch:   4916] loss: 0.13122  | 0.18
[epoch: 4, batch:   4918] loss: 0.65096  | 0.18
[epoch: 4, batch:   4920] loss: 0.07656  | 0.18
[epoch: 4, batch:   4922] loss: 0.32122  | 0.18
[epoch: 4, batch:   4924] loss: 0.01624  | 0.18
[epoch: 4, batch:   4926] loss: 1.07035  | 0.19
[epoch: 4, batch:   4928] loss: 0.04456  | 0.18
[epoch: 4, batch:   4930] loss: 0.02753  | 0.18
[epoch: 4, batch:   4932] loss: 0.40706  | 0.19
[epoch: 4, batch:   4934] loss: 0.13438  | 0.17
[epoch: 4, batch:   4936] loss: 0.04396  | 0.18
[epoch: 4, batch:   4938] loss: 0.06788  | 0.18
[epoch: 4, batch:   4940] loss: 1.83881  | 0.18
[epoch: 4, batch:   4942] loss: 0.05477  | 0.18
[epoch: 4, batch:   4944] loss: 0.56070  | 0.18
[epoch: 4, batch:   4946] loss: 0.50468  | 0.18
[epoch: 4, batch:   4948] loss: 0.02084  | 0.19
[epoch: 4, batch:   4950] loss: 0.07221  | 0.18
[epoch: 4, batch:   4952] loss: 0.04971 

OSError: [Errno 12] Cannot allocate memory

In [6]:
# torch.save(model.state_dict(), 'model.pth')

model.load_state_dict(torch.load('model.pth'))
model.eval()

SomeDrivingModel(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_s

In [ ]:
## freeze all parts except angle regressor

# for param in model.parameters():  # freeze all
#     param.requires_grad = False

# for param in model.control_angle.parameters():  # unfreeze angle regressor
#     param.requires_grad = True

# optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
# criterion2 = nn.SmoothL1Loss()

# for epoch in range(3):
#     running_loss = 0.0
#     start_time = time()
#     for batch_idx, (data, target) in enumerate(train_loader):
#             optimizer.zero_grad()
#             prediction = model(data)
#             loss = criterion2(prediction['canSteering'].cuda(), target['canSteering'].cuda())

#             loss.backward()
#             optimizer.step()

#             # print statistics
#             running_loss += loss.item()
#             if batch_idx % 2 == 1:
#                 cost_time = time() - start_time

#                 print('[epoch: %d, batch:  %5d] loss: %.5f  | %.2f' %
#                       (epoch + 1, batch_idx + 1, running_loss / 2.0, cost_time))

#                 running_loss = 0.0
#                 start_time = time()

#             # Remove this when actually training. 
#             # Used to terminate early. 
#             if batch_idx >= 100: 
#                 break

# for param in model.parameters():  # freeze all
#     param.requires_grad = True

In [15]:
## Local Evaluation


""" undo normalization
"target":{
        "normalize": true,
        "mean": {"canSteering": -5.406788214535221,
                "canSpeed": 13.426163367846936},
        "std": {"canSteering": 73.41232589456718,
                "canSpeed": 7.8257638553586455}
    },
"""


import numpy as np

model.eval()
with torch.no_grad():
    speed_mse = []
    steer_mse = []
    
    for batch_idx, (data, target) in enumerate(validation_loader):
        prediction = model(data)
        # Again only evaluating the canSpeed 
        # predictions, add canSteering when 
        # jointly training.
#         print('pred:', prediction['canSpeed'])
#         print('target:', target['canSpeed'])
        speed_mse.append(
            (
                np.square(
                    (
                        prediction['canSpeed'] * config['target']['std']['canSpeed'] + config['target']['mean']['canSpeed'] 
                        - target['canSpeed'].cuda() * config['target']['std']['canSpeed'] + config['target']['mean']['canSpeed'] 
                    ).cpu()
                )
            ).mean()
        )
    
        steer_mse.append(
            (
                np.square(
                    (
                        prediction['canSteering'] * config['target']['std']['canSteering'] + config['target']['mean']['canSteering'] 
                        - target['canSteering'].cuda() * config['target']['std']['canSteering'] + config['target']['mean']['canSteering'] 
                    ).cpu()
                )
            ).mean()
        )
        
        # Used to terminate early, remove.
#         if batch_idx >= 100: 
#             break

print('Angle MSE: %.2f (872.67), Speed MSE: %.2f (5.766)' 
      % (np.array(steer_mse).mean(), np.array(speed_mse).mean()))

Angle MSE: 1521.36 (872.67), Speed MSE: 889.55 (5.766)


In [ ]:
# torch.save(model.state_dict(), 'model6.pt')

In [9]:
""" create submission file """


normalize_targets = config['target']['normalize']
target_mean = config['target']['mean']
target_std = config['target']['std']

def add_results(results, output):
    steering = np.squeeze(output['canSteering'].cpu().data.numpy())
    speed = np.squeeze(output['canSpeed'].cpu().data.numpy())
    if normalize_targets:
        steering = (steering*target_std['canSteering'])+target_mean['canSteering']
        speed = (speed*target_std['canSpeed'])+target_mean['canSpeed']
    if np.isscalar(steering):
        steering = [steering]
    if np.isscalar(speed):
        speed = [speed]
    results['canSteering'].extend(steering)
    results['canSpeed'].extend(speed)

In [10]:
import pandas as pd

results = {
    'canSteering': [],
    'canSpeed': [],
    'cameraRightFile': []
}

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(test_loader):
        print(batch_idx)
        prediction = model(data)
        add_results(results, prediction)
#         print(target)
        results['cameraRightFile'].append(target['cameraRightFile'][0])

        # Used to terminate early, remove.
#         if batch_idx >= 200: 
#             break
        
# df.to_csv(file, index=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061


3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703


5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347


6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858


In [11]:
## merge with full test file
test3 = pd.DataFrame.from_dict(results)

full_test = pd.read_csv('test_full.csv')
full_test['imageNumber'] = full_test['cameraRight'].str.split('/').str[-1].str[3:8].astype(int)
final = full_test[full_test['imageNumber']>100].copy()

test3['canSteering'] = results['canSteering']
test3['canSpeed'] = results['canSpeed']

final = final.merge(test3, left_on='cameraRight', right_on='cameraRightFile', how='left')

In [12]:
final = final.groupby('chapter').apply(lambda group: group.interpolate(limit_direction='both'))

In [13]:
final.shape

(279863, 9)

In [14]:
from datetime import datetime as dt

file = 'result/submission_%s.csv' % dt.now().strftime('%Y%m%dT%H%M%S')
final[['canSpeed', 'canSteering']].to_csv(file, header=False, index=False)

In [16]:
epoch_error

NameError: name 'epoch_error' is not defined

In [3]:
import sys
print("You are using Python {}.{}.".format(sys.version_info.major, sys.version_info.minor))

You are using Python 3.7.
